<a href="https://colab.research.google.com/github/Dipak22/Case-Studies/blob/master/Transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import classes and functions

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import AlexNet, AlexNet_Weights
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from tqdm import tqdm
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("karakaggle/kaggle-cat-vs-dog-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/kaggle-cat-vs-dog-dataset


In [3]:
PATH_TO_DATA = path +"/kagglecatsanddogs_3367a/PetImages/"
normalizer = transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    normalizer
                                 ])
dataset = ImageFolder(PATH_TO_DATA, transform=train_transforms)
train_samples, test_samples = int(0.9 * len(dataset)), len(dataset) - int(0.9 * len(dataset))
train_dataset, val_dataset = torch.utils.data.random_split(dataset, lengths = [train_samples, test_samples])

##Load model

In [4]:
model = AlexNet()
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [5]:
model.classifier[1]

Linear(in_features=9216, out_features=4096, bias=True)

## update model to change output 2 logits instead of 1000

In [6]:
model = AlexNet()
model.classifier[6] = nn.Linear(in_features=4096, out_features=2, bias=True)
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

### test with dummy data

In [7]:
rand_data = torch.rand(16, 3, 224, 224)
model_output = model(rand_data)
model_output.shape

torch.Size([16, 2])

### count total parameters

In [8]:
total_parameters = 0
for name, param in model.named_parameters():
  num_params = int(torch.prod(torch.tensor(param.shape)))
  print(f"name: {name} , shape: {param.shape} , num_params : {num_params}")
  total_parameters +=num_params
print("--------------------------")
print(f"Total params: {total_parameters}")

name: features.0.weight , shape: torch.Size([64, 3, 11, 11]) , num_params : 23232
name: features.0.bias , shape: torch.Size([64]) , num_params : 64
name: features.3.weight , shape: torch.Size([192, 64, 5, 5]) , num_params : 307200
name: features.3.bias , shape: torch.Size([192]) , num_params : 192
name: features.6.weight , shape: torch.Size([384, 192, 3, 3]) , num_params : 663552
name: features.6.bias , shape: torch.Size([384]) , num_params : 384
name: features.8.weight , shape: torch.Size([256, 384, 3, 3]) , num_params : 884736
name: features.8.bias , shape: torch.Size([256]) , num_params : 256
name: features.10.weight , shape: torch.Size([256, 256, 3, 3]) , num_params : 589824
name: features.10.bias , shape: torch.Size([256]) , num_params : 256
name: classifier.1.weight , shape: torch.Size([4096, 9216]) , num_params : 37748736
name: classifier.1.bias , shape: torch.Size([4096]) , num_params : 4096
name: classifier.4.weight , shape: torch.Size([4096, 4096]) , num_params : 16777216
nam

## Let's train this model which is randomly initialized from scratch

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on device: {device}")

# define model
model = AlexNet()
model.classifier[6] = nn.Linear(4096,2)
model = model.to(device)

#hyper parameters
epochs = 5
optimizer  = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
batch_size = 128

## Build dataloaders
trainloader = DataLoader(train_dataset,
                         batch_size=batch_size,
                         shuffle = True,
                         num_workers=4)
valloader = DataLoader(val_dataset,
                       batch_size=batch_size,
                       shuffle=False,
                       num_workers=4)


Training on device: cuda


### train function

In [10]:
def train(model, device, epochs, optimizer, loss_fn, batch_size, trainloader, valloader):
  log_training = {"epoch":[],
                  "training_loss": [],
                  "training_acc":[],
                  "validation_loss":[],
                  "validation_acc":[]}
  for epoch in range(1, epochs+1):
    training_losses ,training_accuracies = [],[]
    validation_losses, validation_accuracies= [],[]
    model.train()
    for image, label in tqdm(trainloader):
      image, label = image.to(device), label.to(device)
      optimizer.zero_grad()
      out = model.forward(image)
      #calcualte loss
      loss = loss_fn(out, label)
      training_losses.append(loss.item())

      #calculate accuracy
      predictions = torch.argmax(out, dim=1)
      accuracy = (predictions==label).sum()/len(predictions)
      training_accuracies.append(accuracy.item())
      loss.backward()
      optimizer.step()

    model.eval()
    with torch.inference_mode():
      for image, label in tqdm(valloader):
        image, label = image.to(device), label.to(device)
        out = model.forward(image)
        #calculate loss
        loss = loss_fn(out, label)
        validation_losses.append(loss.item())

        #calculate accuracy
        predictions = torch.argmax(out, dim=1)
        accuracy = (predictions==label).sum()/len(predictions)
        validation_accuracies.append(accuracy.item())
    training_loss_mean, training_acc_mean = np.mean(training_losses), np.mean(training_accuracies)
    valid_loss_mean, valid_acc_mean = np.mean(validation_losses), np.mean(validation_accuracies)

    log_training["epoch"].append(epoch)
    log_training["training_loss"].append(training_loss_mean)
    log_training["training_acc"].append(training_acc_mean)
    log_training["validation_loss"].append(valid_loss_mean)
    log_training["validation_acc"].append(valid_acc_mean)

    print("Training Loss:", training_loss_mean)
    print("Training Acc:", training_acc_mean)
    print("Validation Loss:", valid_loss_mean)
    print("Validation Acc:", valid_acc_mean)
  return log_training, model

In [11]:
random_init_logs, model = train(model=model,
                                device=device,
                                epochs=epochs,
                                optimizer=optimizer,
                                loss_fn=loss_fn,
                                batch_size=batch_size,
                                trainloader=trainloader,
                                valloader=valloader)

100%|██████████| 20/20 [00:11<00:00,  1.74it/s]


Training Loss: 0.7107659906826236
Training Acc: 0.5541668077084151
Validation Loss: 0.6372773677110672
Validation Acc: 0.651171875


100%|██████████| 20/20 [00:09<00:00,  2.09it/s]


Training Loss: 0.6217928317121484
Training Acc: 0.6576021093536507
Validation Loss: 0.6067684471607209
Validation Acc: 0.655859375


 24%|██▍       | 43/176 [00:21<01:05,  2.04it/s]


KeyboardInterrupt: 

In [12]:
for image, label in trainloader:
  print(image.shape)
  print(label.shape)
  break

torch.Size([128, 3, 224, 224])
torch.Size([128])


In [15]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
model.classifier[6] = nn.Linear(4096, 2)

# Check the name of all the parameters
for name, param in model.named_parameters():
    if "classifier.6" not in name:
        param.requires_grad_(False) # Inplace turn of gradient updates

model = model.to(device)

### MODEL TRAINING INPUTS ###
epochs = 2
optimizer = optim.Adam(params=model.parameters(), lr=0.0001)
loss_fn = nn.CrossEntropyLoss()
batch_size = 128

### BUILD DATALOADERS ###
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
valloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


random_init_logs, model = train(model=model,
                                device=device,
                                epochs=epochs,
                                optimizer=optimizer,
                                loss_fn=loss_fn,
                                batch_size=batch_size,
                                trainloader=trainloader,
                                valloader=valloader)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
100%|██████████| 20/20 [00:09<00:00,  2.14it/s]


Training Loss: 0.22385637703436342
Training Acc: 0.9072181077843363
Validation Loss: 0.12616125158965588
Validation Acc: 0.954296875


100%|██████████| 20/20 [00:09<00:00,  2.20it/s]

Training Loss: 0.13120953146029601
Training Acc: 0.9497887641191483
Validation Loss: 0.10557032823562622
Validation Acc: 0.9625
